# Задача №2

In [1]:
# импорт библиотек
import pandas as pd
import numpy as np
import os
import copy
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics
# from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
# import xgboost
from sklearn.metrics import accuracy_score

In [2]:
path_to_data = "C:/Users/Pavel/Documents/"

In [3]:
#Считываем данные из файла
df = pd.read_excel(os.path.join(path_to_data, "Task2_TITANIC_DATA.xlsx"),sheet_name = 0)

In [4]:
df

,Класс каюты,Пол,Возраст,Количество родственников на борту,Номер билета,Гражданство,Количество детей на борту,Cтоимость билета,Cпаслись или нет
0,люкс,женский,29.000000,0,3780091001,Великобритания,0,211.3375,1
1,люкс,мужской,0.916700,1,3780091002,Великобритания,2,151.5500,1
2,люкс,женский,2.000000,1,3780091003,Великобритания,2,151.5500,0
3,люкс,мужской,30.000000,1,3780091004,Великобритания,2,151.5500,0
4,люкс,женский,25.000000,1,3780091005,Великобритания,2,151.5500,0
...,...,...,...,...,...,...,...,...,...
1304,эконом,женский,14.500000,1,138911705,Великобритания,0,14.4542,0
1305,эконом,женский,29.881135,1,138911706,Великобритания,0,14.4542,0
1306,эконом,мужской,26.500000,0,138911707,Великобритания,0,7.2250,0
1307,эконом,мужской,27.000000,0,138911708,Великобритания,0,7.2250,0


In [5]:
#Переименовываем столбцы
new_name = ["pclass", "sex", "age", "sibsp","ticket","nationality","parch","fare","survived" ]
df_new_columns = copy.deepcopy(df)
df_new_columns.columns = new_name

In [6]:
df_new_columns.head(10)

,pclass,sex,age,sibsp,ticket,nationality,parch,fare,survived
0,люкс,женский,29.0000,0,3780091001,Великобритания,0,211.3375,1
1,люкс,мужской,0.9167,1,3780091002,Великобритания,2,151.5500,1
2,люкс,женский,2.0000,1,3780091003,Великобритания,2,151.5500,0
3,люкс,мужской,30.0000,1,3780091004,Великобритания,2,151.5500,0
4,люкс,женский,25.0000,1,3780091005,Великобритания,2,151.5500,0
5,люкс,мужской,48.0000,0,3780091006,Великобритания,0,26.5500,1
6,люкс,женский,63.0000,1,3780091007,Великобритания,0,77.9583,1
7,люкс,мужской,39.0000,0,3780091008,Великобритания,0,0.0000,0
8,люкс,женский,53.0000,2,3780091009,Великобритания,0,51.4792,1
9,люкс,мужской,71.0000,0,3780091010,Великобритания,0,49.5042,0


In [7]:
df_new_columns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   pclass       1309 non-null   object 
 1   sex          1309 non-null   object 
 2   age          1309 non-null   float64
 3   sibsp        1309 non-null   int64  
 4   ticket       1309 non-null   int64  
 5   nationality  1294 non-null   object 
 6   parch        1309 non-null   int64  
 7   fare         1309 non-null   float64
 8   survived     1309 non-null   int64  
dtypes: float64(2), int64(4), object(3)
memory usage: 92.2+ KB


In [8]:
#Удаляем столбец номер билета, так как он вряд ли имеет предсказательную силу
df_new_columns.drop(['ticket','nationality'], axis=1, inplace=True)

In [9]:
#Вводим дополнительные расчетные столбцы / удаляем уже ненужные фичи
df_new_columns['family']  = df_new_columns['parch'] + df_new_columns['sibsp']
df_new_columns.drop(['parch','sibsp'], axis=1, inplace=True)
df_new_columns['is_alone'] = np.where(df_new_columns.family == 0, 1, 0)

In [10]:
#Кодируем фичи
dict_pclass={'люкс': 1,'комфорт': 2,'эконом': 3 }
dict_sex={'мужской': 1,'женский': 2}
df_new_columns['pclass'] = df_new_columns['pclass'].map(dict_pclass)
df_new_columns['sex'] = df_new_columns['sex'].map(dict_sex)

In [11]:
df_new_columns.head(10)

,pclass,sex,age,fare,survived,family,is_alone
0,1,2,29.0000,211.3375,1,0,1
1,1,1,0.9167,151.5500,1,3,0
2,1,2,2.0000,151.5500,0,3,0
3,1,1,30.0000,151.5500,0,3,0
4,1,2,25.0000,151.5500,0,3,0
5,1,1,48.0000,26.5500,1,0,1
6,1,2,63.0000,77.9583,1,1,0
7,1,1,39.0000,0.0000,0,0,1
8,1,2,53.0000,51.4792,1,2,0
9,1,1,71.0000,49.5042,0,0,1


In [12]:
#Разбиваем возраст пассажиров Титаника на категории
cut_labels_age = [1, 2, 3, 4, 5,6, 7,8]
cut_bins_age = [0, 10, 20, 30, 40, 50, 60, 70, 80]
df_new_columns['age_'] = pd.cut(df_new_columns['age'], bins=cut_bins_age, labels=cut_labels_age)
df_new_columns.drop(['age'], axis=1, inplace=True)
df_new_columns.rename(columns={'age_': 'age'}, inplace=True)

In [13]:
df_new_columns.head(10)

,pclass,sex,fare,survived,family,is_alone,age
0,1,2,211.3375,1,0,1,3
1,1,1,151.5500,1,3,0,1
2,1,2,151.5500,0,3,0,1
3,1,1,151.5500,0,3,0,3
4,1,2,151.5500,0,3,0,3
5,1,1,26.5500,1,0,1,5
6,1,2,77.9583,1,1,0,7
7,1,1,0.0000,0,0,1,4
8,1,2,51.4792,1,2,0,6
9,1,1,49.5042,0,0,1,8


In [14]:
#Разбиваем тариф пассажиров Титаника на категории, перед этим избавимся от нулевых значений, заменив их средними по классу
mean_fare_pclass1 = round(df_new_columns[(df_new_columns['pclass']==1)&(df_new_columns['fare']!=0)]['fare'].mean(),2)
mean_fare_pclass2 = round(df_new_columns[(df_new_columns['pclass']==2)&(df_new_columns['fare']!=0)]['fare'].mean(),2)
mean_fare_pclass3 = round(df_new_columns[(df_new_columns['pclass']==3)&(df_new_columns['fare']!=0)]['fare'].mean(),2)

df_new_columns.loc[(df_new_columns['pclass']==1)&(df_new_columns['fare']==0), ['fare']] = mean_fare_pclass1
df_new_columns.loc[(df_new_columns['pclass']==2)&(df_new_columns['fare']==0), ['fare']] = mean_fare_pclass2
df_new_columns.loc[(df_new_columns['pclass']==3)&(df_new_columns['fare']==0), ['fare']] = mean_fare_pclass3

cut_labels_fare = [1, 2, 3, 4, 5,6, 7,8,9,10,11]
cut_bins_fare = [0, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500,550]
df_new_columns['fare_'] = pd.cut(df_new_columns['fare'], bins=cut_bins_fare, labels=cut_labels_fare)
df_new_columns.drop(['fare'], axis=1, inplace=True)
df_new_columns.rename(columns={'fare_': 'fare'}, inplace=True)

In [15]:
df_new_columns.head(10)

,pclass,sex,survived,family,is_alone,age,fare
0,1,2,1,0,1,3,5
1,1,1,1,3,0,1,4
2,1,2,0,3,0,1,4
3,1,1,0,3,0,3,4
4,1,2,0,3,0,3,4
5,1,1,1,0,1,5,1
6,1,2,1,1,0,7,2
7,1,1,0,0,1,4,2
8,1,2,1,2,0,6,2
9,1,1,0,0,1,8,1


In [16]:
#Разбиваем данные на учебную и тестовую выборку
X = df_new_columns.drop(['survived'], axis=1)
y = df_new_columns['survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=17)

In [17]:
%%time
# Тестируем 4 алгоритма
list_alg = [GaussianNB(), KNeighborsClassifier(), DecisionTreeClassifier(), RandomForestClassifier(n_estimators=200)]
list_name = ['Наивный Байес', 'K-ближайших соседей', 'Деревья решений', 'Случайный лес', 'XGBoost']
for alg, name_alg in zip(list_alg, list_name):
    model = alg
    model.fit(X_train, y_train)
    expected = y_test
    predicted = model.predict(X_test)

    # Оценка качества алгоритма
    print('--------------- ' + name_alg + ' ---------------')
    print(metrics.classification_report(expected, predicted))
    print(metrics.confusion_matrix(expected, predicted))

    y_train_predict = model.predict(X_train)
    y_test_predict = model.predict(X_test)
    err_train = np.mean(y_train != y_train_predict)
    err_test = np.mean(y_test != y_test_predict)
    print(round(err_train,2), round(err_test,2))

    accuracy = accuracy_score(y_test, y_test_predict)
    print("Accuracy: %.2f%%" % (accuracy * 100.0))

--------------- Наивный Байес ---------------
              precision    recall  f1-score   support

           0       0.82      0.81      0.82       243
           1       0.71      0.72      0.71       150

    accuracy                           0.78       393
   macro avg       0.77      0.77      0.77       393
weighted avg       0.78      0.78      0.78       393

[[198  45]
 [ 42 108]]
0.21 0.22
Accuracy: 77.86%
--------------- K-ближайших соседей ---------------
              precision    recall  f1-score   support

           0       0.82      0.85      0.83       243
           1       0.74      0.70      0.72       150

    accuracy                           0.79       393
   macro avg       0.78      0.77      0.78       393
weighted avg       0.79      0.79      0.79       393

[[206  37]
 [ 45 105]]
0.18 0.21
Accuracy: 79.13%
--------------- Деревья решений ---------------
              precision    recall  f1-score   support

           0       0.81      0.86      0.84  

In [18]:
%%time
# Подбор оптимальных показателей алгоритма
# Создаем словарь с параметрами, которые хотим проверить
params = {"n_estimators": [200], "max_depth": range(5, 10), "min_samples_split": range(5, 10),
          "min_samples_leaf": range(5, 10)}
# Cоздаем экземпляр GridSearchCV
model = RandomForestClassifier()
search = GridSearchCV(model, params, cv=5, n_jobs=-1)
# Обучаем дерево
search.fit(X_train, y_train)
# Сохраняем лучшее дерево по мнению GridSearchCV
best_tree = search.best_estimator_
expected = y_test
predicted = best_tree.predict(X_test)
# Оценка качества алгоритма
print('--------------- Случайный лес / Лучшие параметры ---------------')
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

y_train_predict = best_tree.predict(X_train)
y_test_predict = best_tree.predict(X_test)
err_train = np.mean(y_train != y_train_predict)
err_test = np.mean(y_test != y_test_predict)
print(err_train, err_test)

--------------- Случайный лес / Лучшие параметры ---------------
              precision    recall  f1-score   support

           0       0.83      0.88      0.85       243
           1       0.78      0.71      0.75       150

    accuracy                           0.81       393
   macro avg       0.81      0.79      0.80       393
weighted avg       0.81      0.81      0.81       393

[[213  30]
 [ 43 107]]
0.18777292576419213 0.18575063613231552
Wall time: 2min
